# Face Detection

In this notebook we will run a face detection model to detect faces in the camera feed.

## Import modules

In [1]:
import cv2
import sys
import numpy

## Set the model parameters

In [2]:
# Model parameters
in_width = 300
in_height = 300
mean = [104, 117, 123]
conf_threshold = 0.7

## Set source

Set source as `0` or the `index` of the webcam to be used if you have multiple webcams.

In [3]:
source = 0

## Initialize the model object

We will use a pre-trained caffe model to detect the face.

In [4]:
net = cv2.dnn.readNetFromCaffe("../../data/deploy.prototxt",
                               "../../data/res10_300x300_ssd_iter_140000_fp16.caffemodel")

## Initialize the capture object and preview window

In [5]:
cap = cv2.VideoCapture(source)
win_name = 'Camera Preview'
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)

## Capture each frame, run detection and preview

In [6]:
while cv2.waitKey(1) != 27:
    has_frame, frame = cap.read()
    if not has_frame:
        break

    frame_height = frame.shape[0]
    frame_width = frame.shape[1]

    # Create a 4D blob from a frame.
    blob = cv2.dnn.blobFromImage(frame, 1.0, (in_width, in_height), mean, False, False)

    # Run a model
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x_left_bottom = int(detections[0, 0, i, 3] * frame_width)
            y_left_bottom = int(detections[0, 0, i, 4] * frame_height)
            x_right_top = int(detections[0, 0, i, 5] * frame_width)
            y_right_top = int(detections[0, 0, i, 6] * frame_height)

            cv2.rectangle(frame, (x_left_bottom, y_left_bottom), (x_right_top, y_right_top), (0, 255, 0))
            label = "Confidence: %.4f" % confidence
            label_size, base_line = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)

            cv2.rectangle(frame, (x_left_bottom, y_left_bottom - label_size[1]),
                                (x_left_bottom + label_size[0], y_left_bottom + base_line),
                                (255, 255, 255), cv2.FILLED)
            cv2.putText(frame, label, (x_left_bottom, y_left_bottom),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

    t, _ = net.getPerfProfile()
    label = 'Inference time: %.2f ms' % (t * 1000.0 / cv2.getTickFrequency())
    cv2.putText(frame, label, (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))

    cv2.imshow(win_name, frame)
cv2.destroyWindow(win_name)

## Release capture device

In [7]:
cap.release()

## Thank You!